In [22]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from dash import Dash, dcc, html, dash_table
import plotly.express as px

def extract_all_high_temperatures():
    # URL for NYC's weather forecast
    url = "https://forecast.weather.gov/MapClick.php?lat=40.7128&lon=-74.0060"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find all high temperature texts using regex
    page_text = soup.get_text()
    high_temp_matches = re.findall(r"High:\s*(\d+)\s*°F", page_text)
    
    # Convert matches to integers
    high_temps = [int(temp) for temp in high_temp_matches]
    return high_temps

# Test the function
high_temperatures = extract_all_high_temperatures()

# Step 2: Prepare Data for the Dashboard
# Simulating timestamps for visualization purposes
dates = pd.date_range(start="2024-11-28", periods=len(high_temperatures), freq="D")
data = pd.DataFrame({"Date": dates, "High Temperature (°F)": high_temperatures})

# Step 3: Build the Dashboard
app = Dash(__name__)

app.layout = html.Div([
    html.H1("NYC High Temperature Forecast", style={'textAlign': 'center'}),
    
    dash_table.DataTable(
        id='high-temp-table',
        columns=[
            {"name": "Date", "id": "Date", "type": "datetime"},
            {"name": "High Temperature (°F)", "id": "High Temperature (°F)", "type": "numeric"}
        ],
        data=data.to_dict("records"),  # Convert DataFrame to a format DataTable understands
        style_table={'margin': '0 auto', 'width': '60%'},  # Center and adjust table width
        style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'},
        style_cell={'textAlign': 'center', 'padding': '5px'},
        style_data_conditional=[
            {
                'if': {'filter_query': '{High Temperature (°F)} >= 50', 'column_id': 'High Temperature (°F)'},
                'backgroundColor': 'tomato',
                'color': 'white',
                'fontWeight': 'bold'
            },
            {
                'if': {'filter_query': '{High Temperature (°F)} < 50', 'column_id': 'High Temperature (°F)'},
                'backgroundColor': 'lightgreen',
                'color': 'black'
            }
        ],
        sort_action="native",  # Enable sorting
        filter_action="native"  # Enable filtering
    )
])

# Step 4: Run the Dashboard
if __name__ == "__main__":
    app.run_server(debug=True)
